# Upgrade to rclone-based Storage Initializer - automation for AWS S3 / MinIO configuration

In this documentation page we will provide an example upgrade path from kfserving-based to rclone-based storage initializer.

Storage initializers are used by Seldon's pre-packaged model servers to download models binaries. 
As it is explained in the [SC 1.8 upgrading notes](https://docs.seldon.io/projects/seldon-core/en/latest/reference/upgrading.html#upgrading-to-1-8) the `seldonio/rclone-storage-initializer` became default storage initializer in v1.8.0.


In this tutorial we will show how to upgrade your configuration to new Storage Initializer with focus on automating the whole process to the extent that is possible.

Read more on [Prepackaged Model Servers](https://docs.seldon.io/projects/seldon-core/en/latest/servers/overview.html) documentation page.

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * mc client
 * curl

## Steps in this tutorial
 
 * Start with SC configured to use kfserving-based storage initializer
 * Copy iris model from GCS into in-cluster minio
 * Deploy SKlearn Pre-Packaged server using kfserving storage initializer
     A) Providing credentials using old-style storage initializer secret
     B) Providing credentials using old-style storage initializer Service Account format
 * Extend secrets to include rclone-specific fields (patch Seldon Deployments where required)
 * Upgrade SC installation to use rclone-based storage initializer
 
## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). 

Set starting storage initializer to be kfserving one

In [1]:
%%bash
helm upgrade seldon-core seldon-core-operator \
    --install \
    --repo https://storage.googleapis.com/seldon-charts \
    --version 1.9.1 \
    --namespace seldon-system \
    --set storageInitializer.image="gcr.io/kfserving/storage-initializer:v0.4.0" \
    --reuse-values

Release "seldon-core" has been upgraded. Happy Helming!
NAME: seldon-core
LAST DEPLOYED: Tue Jun 29 22:49:39 2021
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 16
TEST SUITE: None


index.go:339: skipping loading invalid entry for chart "seldon-core" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.5a" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.5a" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core" "0.1.6_SNAPSHOT" from /home/rskolasinski/.cache/helm/repository/LQ7rLhE5lxPEzQjIRiXKshmBgxM=-index.yaml: validation: chart.metadata.ver

## Setup MinIO

Use the provided [notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html) to install Minio in your cluster and configure `mc` CLI tool. 

## Copy iris model into local MinIO

In [2]:
%%bash
mc config host add gcs https://storage.googleapis.com "" "" 

mc mb minio-seldon/sklearn/iris/ -p
mc cp gcs/seldon-models/sklearn/iris/model.joblib minio-seldon/sklearn/iris/
mc cp gcs/seldon-models/sklearn/iris/metadata.yaml minio-seldon/sklearn/iris/

Added `gcs` successfully.
Bucket created successfully `minio-seldon/sklearn/iris/`.
`gcs/seldon-models/sklearn/iris/model.joblib` -> `minio-seldon/sklearn/iris/model.joblib`
Total: 0 B, Transferred: 1.06 KiB, Speed: 1.72 KiB/s
`gcs/seldon-models/sklearn/iris/metadata.yaml` -> `minio-seldon/sklearn/iris/metadata.yaml`
Total: 0 B, Transferred: 162 B, Speed: 326 B/s


In [3]:
%%bash
mc ls minio-seldon/sklearn/iris/

[2021-06-29 22:49:49 BST]    162B metadata.yaml
[2021-06-29 22:49:47 BST]  1.1KiB model.joblib


## Deploy SKLearn Server with kfserving-storage-initializer

First we deploy the model using kfserving-storage-initializer. This is using the default Storage Initializer for pre Seldon Core v1.8.0.

## Using envSecretRefName

In [4]:
%%writefile sklearn-iris-secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: seldon-kfserving-secret
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: minioadmin
  AWS_SECRET_ACCESS_KEY: minioadmin
  AWS_ENDPOINT_URL: http://minio.minio-system.svc.cluster.local:9000
  USE_SSL: "false"
    
---
    
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn-iris-secret
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: s3://sklearn/iris
      envSecretRefName: seldon-kfserving-secret

Overwriting sklearn-iris-secret.yaml


In [5]:
!kubectl apply -f sklearn-iris-secret.yaml

secret/seldon-kfserving-secret configured
seldondeployment.machinelearning.seldon.io/sklearn-iris-secret unchanged


In [6]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-secret -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-iris-secret-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-iris-secret-default-0-classifier" successfully rolled out


In [7]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/sklearn-iris-secret/api/v1.0/predictions  | jq .

## Using serviceAccountName

In [17]:
%%writefile sklearn-iris-sa.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
secrets:
  - name: minio-sa-secret

---

apiVersion: v1
kind: Secret
metadata:
  name: minio-sa-secret
  annotations:
     machinelearning.seldon.io/s3-endpoint: minio.minio-system.svc.cluster.local:9000
     machinelearning.seldon.io/s3-usehttps: "0"
type: Opaque
stringData:
  awsAccessKeyID: "minioadmin"
  awsSecretAccessKey: "minioadmin"

---
    
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn-iris-sa
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: s3://sklearn/iris
      serviceAccountName: minio-sa

Overwriting sklearn-iris-sa.yaml


In [18]:
!kubectl apply -f sklearn-iris-sa.yaml

serviceaccount/minio-sa configured
secret/minio-sa-secret configured
seldondeployment.machinelearning.seldon.io/sklearn-iris-sa configured


In [19]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-sa -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-iris-sa-default-0-classifier" successfully rolled out


In [20]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/sklearn-iris-sa/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364169,
        0.04505474761561406,
        5.7927447968952436e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.10.0-dev"
    }
  }
}


## Preparing rclone-compatible secret

The [rclone](https://rclone.org/)-based storage initializer expects one to define a new secret. General documentation credentials hadling can be found [here](https://docs.seldon.io/projects/seldon-core/en/latest/servers/overview.html#handling-credentials) with constantly updated examples of tested configurations.

If we do not have yet an example for Cloud Storage solution that you are using, please, consult the relevant page on [RClone documentation](https://rclone.org/#providers).

## Updating envSecretRefName-specified secrets

In [46]:
from kubernetes import client, config
from typing import List, Tuple


AWS_SECRET_REQUIRED_FIELDS = [
    'AWS_ACCESS_KEY_ID', 'AWS_ENDPOINT_URL', 'AWS_SECRET_ACCESS_KEY'
]


def get_secrets_to_update(namespace: str) -> List[str]:
    """Get list of secrets defined for Seldon Deployments in a given namespace.
    
    Parameters:
    ----------
    namespace: str
        Namespace in which to look for secrets attached to Seldon Deployments.
        
    Returns:
    -------
    secrets_names: List[str]
        List of secrets names
    """
    secret_names = []
    api_instance = client.CustomObjectsApi()
    sdeps = api_instance.list_namespaced_custom_object(
        "machinelearning.seldon.io",
        "v1",
        namespace,
        "seldondeployments",
    )
    for sdep in sdeps.get("items", []):
        for predictor in sdep.get("spec", {}).get("predictors", []):
            secret_name = predictor.get("graph", {}).get("envSecretRefName", None)
            if secret_name:
                secret_names.append(secret_name)
    return secret_names


def new_fields_for_secret(secret, provider):
    for key in AWS_SECRET_REQUIRED_FIELDS:
        if key not in secret.data:
            raise ValueError(f"Secret '{secret.metadata.name}' does not contain '{key}' field.")
    
    return {
        "data": {
            "RCLONE_CONFIG_S3_ACCESS_KEY_ID": secret.data.get("AWS_ACCESS_KEY_ID"),
            "RCLONE_CONFIG_S3_SECRET_ACCESS_KEY": secret.data.get("AWS_SECRET_ACCESS_KEY"),
            "RCLONE_CONFIG_S3_ENDPOINT": secret.data.get("AWS_ENDPOINT_URL"),
        },
        "stringData": {
            "RCLONE_CONFIG_S3_TYPE": "s3",
            "RCLONE_CONFIG_S3_PROVIDER": provider,
            "RCLONE_CONFIG_S3_ENV_AUTH": "false",
        }
    }


def update_aws_secrets(namespaces=[], provider="minio"):
    v1 = client.CoreV1Api()
    for namespace in namespaces:
        print(f"Updating secrets in namespace {namespace}")
        secret_names = get_secrets_to_update(namespace)
        for secret_name in secret_names:
            secret = v1.read_namespaced_secret(secret_name, namespace)
            try:
                new_fields = new_fields_for_secret(secret, provider)
            except ValueError as e:
                print(f"  Couldn't upgrade a secret: {e}.")
                continue
            _ = v1.patch_namespaced_secret(
                secret_name, 
                namespace, 
                client.V1Secret(
                    data=new_fields["data"], 
                    string_data=new_fields["stringData"]
                )
            )
            print(f"  Upgraded secret {secret_name}.")

In [47]:
config.load_kube_config()
update_aws_secrets(namespaces=["seldon"], provider="minio")

Updating secrets in namespace seldon
  Couldn't upgrade a secret: Secret 'minio-sa-secret' does not contain 'AWS_ACCESS_KEY_ID' field..
  Upgraded secret seldon-kfserving-secret.


### Updating serviceAccountName-specified secrets and deployments

In [49]:
AWS_SA_SECRET_REQUIRED_FIELDS = [
    "awsAccessKeyID", 
    "awsSecretAccessKey"
]

AWS_SA_SECRET_REQUIRED_ANNOTATIONS = [
    "machinelearning.seldon.io/s3-usehttps",
    "machinelearning.seldon.io/s3-endpoint",
]


def get_sdeps_with_service_accounts(namespace: str) -> List[Tuple[dict, List[str]]]:
    """Get list of secrets defined for Seldon Deployments in a given namespace.
    
    Parameters:
    ----------
    namespace: str
        Namespace in which to look for secrets attached to Seldon Deployments.
        
    Returns:
    -------
    output: List[Tuple[dict, List[dict]]]]
        Eeach tuple contain sdep (dict) and a list service account names (str)
        The list of Service Account names is of length of number of predictors.
        If Predictor has no related Service Account a None is included.
    """
    output = []
    api_instance = client.CustomObjectsApi()
    sdeps = api_instance.list_namespaced_custom_object(
        "machinelearning.seldon.io",
        "v1",
        namespace,
        "seldondeployments",
    )
    for sdep in sdeps.get("items", []):
        sa_names = []
        for predictor in sdep.get("spec", {}).get("predictors", []):
            sa_name = predictor.get("graph", {}).get("serviceAccountName", None)
            sa_names.append(sa_name)
        output.append((sdep, sa_names))
    return output


def find_sa_related_secret(sa_name, namespace):
    v1 = client.CoreV1Api()
    service_account = v1.read_namespaced_service_account(sa_name, namespace)
    for s in service_account.secrets:
        secret = v1.read_namespaced_secret(s.name, namespace)
        if not all(key in secret.data for key in AWS_SA_SECRET_REQUIRED_FIELDS):
            continue
        if not all (key in secret.metadata.annotations for key in AWS_SA_SECRET_REQUIRED_ANNOTATIONS):
            continue
        return secret
    return None


def new_field_for_sa_secret(secret, provider):
    for key in AWS_SA_SECRET_REQUIRED_FIELDS:
        if key not in secret.data:
            raise ValueError(f"Secret '{secret.metadata.name}' does not contain '{key}' field.")    

    use_https = secret.metadata.annotations.get('machinelearning.seldon.io/s3-usehttps', None)
    if use_https == "0":
        protocol = "http"
    elif use_https == "1":
        protocol = "https"
    else:
        raise ValueError(f"Cannot determine http(s) protocol for {secret.metadata.name}.")

    s3_endpoint = secret.metadata.annotations.get('machinelearning.seldon.io/s3-endpoint', None)
    if s3_endpoint is None:
        raise ValueError(f"Cannot determine S3 endpoint for {secret.metadata.name}.")

    endpoint = f"{protocol}://{s3_endpoint}"

    return {
        "data": {
            "RCLONE_CONFIG_S3_ACCESS_KEY_ID": secret.data.get("awsAccessKeyID"),
            "RCLONE_CONFIG_S3_SECRET_ACCESS_KEY": secret.data.get("awsSecretAccessKey"),
        },
        "stringData": {
            "RCLONE_CONFIG_S3_TYPE": "s3",
            "RCLONE_CONFIG_S3_PROVIDER": provider,
            "RCLONE_CONFIG_S3_ENV_AUTH": "false",
            "RCLONE_CONFIG_S3_ENDPOINT": endpoint,            
        }    
    } 


def update_aws_sa_resources(namespaces, provider):
    v1 = client.CoreV1Api()
    api_instance = client.CustomObjectsApi()    
    for namespace in namespaces:
        for sdep, sa_names_per_predictor in get_sdeps_with_service_accounts(namespace):
            update_body = {"spec": sdep["spec"]}
            for n, sa_name in enumerate(sa_names_per_predictor):
                if sa_name is None:
                    continue
                secret = find_sa_related_secret(sa_name, namespace)
                if secret is None:
                    print(f"Couldn't find secret with S3 credentials for {sa.metadata.name}")
                    continue
                new_fields = new_field_for_sa_secret(secret, "minio")
                _ = v1.patch_namespaced_secret(
                    secret.metadata.name, 
                    namespace, 
                    client.V1Secret(
                        data=new_fields["data"], 
                        string_data=new_fields["stringData"]
                    )
                )     
                update_body["spec"]["predictors"][n]["graph"]["envSecretRefName"] = secret.metadata.name
            api_instance.patch_namespaced_custom_object(
                "machinelearning.seldon.io",
                "v1",
                namespace,
                "seldondeployments",
                sdep["metadata"]["name"],    
                update_body,
            )

In [50]:
update_aws_sa_resources(namespaces=["seldon"], provider="minio")

## Upgrade Seldon Core to use new storage initializer

In [51]:
%%bash
helm upgrade seldon-core seldon-core-operator \
    --install \
    --repo https://storage.googleapis.com/seldon-charts \
    --version 1.9.1 \
    --namespace seldon-system \
    --set storageInitializer.image="seldonio/rclone-storage-initializer:1.9.1" \
    --reuse-values

Release "seldon-core" has been upgraded. Happy Helming!
NAME: seldon-core
LAST DEPLOYED: Tue Jun 29 23:15:42 2021
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 18
TEST SUITE: None


index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.5a" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.5a" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core" "0.1.6_SNAPSHOT" from https://storage.googleapis.com/seldon-charts: validation: chart.metadata.version "0.1.6_SNAPSHOT" is invalid
index.go:339: skipping loading invalid entry for chart "seldon-core-crd" "0.1.5a" from /home/rskolasinski/.cache/helm/repository/Uvbs1VeMcTebAWeF8Qc-ZQ9cQlI=-index.yaml: validation: chart.metadata.version

In [58]:
%%bash
kubectl rollout restart -n seldon-system deployments/seldon-controller-manager
kubectl rollout status -n seldon-system deployments/seldon-controller-manager

deployment.apps/seldon-controller-manager restarted
Waiting for deployment "seldon-controller-manager" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "seldon-controller-manager" rollout to finish: 1 old replicas are pending termination...
deployment "seldon-controller-manager" successfully rolled out


In [59]:
from time import sleep
sleep(10)

In [60]:
%%bash

kubectl rollout restart deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-secret -o jsonpath='{.items[0].metadata.name}')
kubectl rollout restart deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-sa -o jsonpath='{.items[0].metadata.name}')

kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-secret -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-sa -o jsonpath='{.items[0].metadata.name}')

deployment.apps/sklearn-iris-secret-default-0-classifier restarted
deployment.apps/sklearn-iris-sa-default-0-classifier restarted
Waiting for deployment "sklearn-iris-secret-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "sklearn-iris-secret-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
deployment "sklearn-iris-secret-default-0-classifier" successfully rolled out
deployment "sklearn-iris-sa-default-0-classifier" successfully rolled out


In [62]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/sklearn-iris-secret/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364169,
        0.04505474761561406,
        5.7927447968952436e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.10.0-dev"
    }
  }
}


In [63]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/sklearn-iris-sa/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364169,
        0.04505474761561406,
        5.7927447968952436e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.10.0-dev"
    }
  }
}


## Cleanup

In [67]:
%%bash
kubectl delete -f sklearn-iris-sa.yaml || echo "already removed"
kubectl delete -f sklearn-iris-secret.yaml || echo "already removed"

already removed
already removed


Error from server (NotFound): error when deleting "sklearn-iris-sa.yaml": serviceaccounts "minio-sa" not found
Error from server (NotFound): error when deleting "sklearn-iris-sa.yaml": secrets "minio-sa-secret" not found
Error from server (NotFound): error when deleting "sklearn-iris-sa.yaml": seldondeployments.machinelearning.seldon.io "sklearn-iris-sa" not found
Error from server (NotFound): error when deleting "sklearn-iris-secret.yaml": secrets "seldon-kfserving-secret" not found
Error from server (NotFound): error when deleting "sklearn-iris-secret.yaml": seldondeployments.machinelearning.seldon.io "sklearn-iris-secret" not found
